<a href="https://colab.research.google.com/github/Josh-Been/Freedom-Narratives-Deuteronomy/blob/master/fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[PPTX](https://baylor.box.com/s/v31v722ueyvag6l8hnvi7nii02pes57z)

Specify Input Files

In [0]:
from google.colab import files
from google.colab import output
doc_files={}

def read_text(type):
    # Browse/Upload File
    up=files.upload()
    # File passed to variable doc
    doc=next(iter(up))
    if type==1:
        try:
            f=open(doc,'r', encoding='utf-8-sig')
            txt=f.read()
            f.close()
        except:
            f=open(doc,'r', encoding='ISO-8859-1')
            txt=f.read()
            f.close()
        return doc, txt
    else:
        import zipfile
        with zipfile.ZipFile(doc, 'r') as zip_ref:
            zip_ref.extractall()
        for txt in zip_ref.namelist():
            if txt.endswith('.txt'):
                if txt.endswith('.txt'):
                    try:
                        f=open(txt, 'r', encoding='utf-8-sig')
                        doc_files[txt]=f.read()
                        f.close()
                    except:
                        f=open(txt, 'r', encoding='ISO-8859-1')
                        doc_files[txt]=f.read()
                        f.close()
        return doc_files

print('(1) I want to upload a source and target text files to compare.')
print('(2) I want to upload a single zip file to comnpare all archived text files')
file_choice=input('Enter 1 or 2\n')

if file_choice=='1':
    print('First, browse for source text file')
    source, source_text=read_text(1)
    print('\nSecond, browse for source text file')
    target, target_text=read_text(1)
    doc_files[source]=source_text
    doc_files[target]=target_text
elif file_choice=='2':
    print('Browse for .zip file\nMake sure only text files are archived')
    doc_files=read_text(2)

Specify:


1.   Ngram Size
2.   Fuzzy Method
3.   Minimuim Match Score
4.   Speed vs. Accuracy



In [0]:
#################################
ngram_size=10
fuzzy_method='token sort ratio'  #[ratio, partial ratio, token sort ratio, token set ratio, exact]
minimum_score=70
speed=3    # 1 is slowest, represents the nth ngrams checked in the target file. 1 is every ngram. 2 is every other ngram. 3 is every third ngram.
#################################

if fuzzy_method!='exact':
    !pip install python-Levenshtein
    from google.colab import output
    try:
        from fuzzywuzzy import fuzz
    except:
        !pip install fuzzywuzzy
        from fuzzywuzzy import fuzz
output.clear()

import csv, re, math

def all_pairs(source):
    result = []
    for p1 in range(len(source)):
        for p2 in range(p1+1,len(source)):
            result.append([source[p1],source[p2]])
    return result

def strip_non_ascii(cleanme):
    cleanme = re.sub(r'[^\w\s]','',cleanme)
    cleanme = cleanme.replace('\n',' ')
    cleanme = cleanme.replace('\r',' ')
    cleanme = cleanme.lower()
    cleanme = ' '.join([word for word in cleanme.split() if not 'page' in word and not word.isdigit()])
    return ''.join([i if ord(i) < 128 else ' ' for i in cleanme])

def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(' '.join(input[i:i+n]))
    return output

def roundup(x):
    return int(math.ceil(x / 100.0)) * 100

matches=['DOC1_TITLE,DOC1_NGRAM,DOC2_TITLE,DOC2_NGRAM,SCORE,MATCH_METHOD\n']
used_keys=[]

for x in all_pairs(list(doc_files.keys())):
    source_ngrams=(ngrams(strip_non_ascii(doc_files[x[0]]),ngram_size))
    target_ngrams=(ngrams(strip_non_ascii(doc_files[x[1]]),ngram_size))
    i=0
    skip=0
    for s in source_ngrams:
        skip=skip-1
        i+=1
        if i==1 or i%100==0:
            print('\n'+x[0]+'\n'+x[1]+'\nChecking Ngram '+str(i)+'-'+str(roundup(i+99))+'/'+str(len(source_ngrams)))
        if skip <=0:
            for target_skip in range(0,len(target_ngrams)):
                if target_skip==0 or target_skip%speed==0:
                    measure=0
                    if fuzzy_method=='ratio':
                        measure=fuzz.ratio(s,target_ngrams[target_skip])
                    elif fuzzy_method=='partial ratio':
                        measure=fuzz.partial_ratio(s,target_ngrams[target_skip])
                    elif fuzzy_method=='token sort ratio':
                        measure=fuzz.token_sort_ratio(s,target_ngrams[target_skip])
                    elif fuzzy_method=='token set ratio':
                        measure=fuzz.token_set_ratio(s,target_ngrams[target_skip])
                    ### Adding exact
                    elif s==target_ngrams[target_skip]:
                        measure=100
                    ###
                    if measure>=minimum_score:
                        matches.append(x[0].replace(',','')+','+s.replace(',','')+','+x[1].replace(',','')+','+target_ngrams[target_skip].replace(',','')+','+str(measure)+','+fuzzy_method+'\n')
                        print(measure, '  >>  ', s, '|', target_ngrams[target_skip])
                        skip=ngram_size/2 

f=open('similarities.csv','w', encoding='utf-8-sig')
for match in matches:
    f.write(match)
f.close()

print('\nCompleted!')